In [ ]:
---
title: "Non linear programming in pyomo"
author: "cbarz"
date: "2022-xx-yy"
categories: [pyomo, non linear programming]
draft: true
code-fold: true
toc: true
description: "basic pyomo workflows for nlp"
---

SyntaxError: invalid syntax (<ipython-input-1-b5b7617975fa>, line 1)

pyomo reference: https://pyomo.readthedocs.io/en/stable/contributed_packages/mindtpy.html

With the mixed-integer nonlinear decomposition toolbox in Pyomo `MindtPy` minlps can be solved using the following decomposition algorithems:

1. [Outer-apporximation](https://dx.doi.org/10.1007/BF02592064)
1. [LP/NLP based branch and bound](https://www.sciencedirect.com/science/article/abs/pii/0098135492800288)
1. [extended cutting plane](http://dx.doi.org/10.1016/0098-1354(95)87027-X)
1. [global outer approximation](https://link.springer.com/article/10.1007/s10107-004-0503-1)
1. [regularized outer approximation](https://link.springer.com/article/10.1007%2Fs10107-018-1356-3)
1. [feasibility pump](https://link.springer.com/article/10.1007/s10107-008-0212-2)

main buidling blocks :
1. instanciate model `ConcreteModel()`
1. add sets
1. add variables
1. add constraints
1. add objective
1. define solver and solve model

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

model.x = Var(bounds=(1.0,10.0),initialize=5.0)
model.y = Var(within=Binary)

model.c1 = Constraint(expr=(model.x-4.0)**2 - model.x <= 50.0*(1-model.y))
model.c2 = Constraint(expr=model.x*log(model.x)+5.0 <= 50.0*(model.y))

model.objective = Objective(expr=model.x, sense=minimize)

SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver='ipopt') 

{'Problem': [{'Name': 'unknown', 'Lower bound': 2.43844718719117, 'Upper bound': 2.4384471872707754, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of binary variables': 1, 'Number of integer variables': 0, 'Number of continuous variables': 1, 'Number of nonzeros': None, 'Sense': 'minimize', 'Number of disjunctions': 0}], 'Solver': [{'Name': 'MindtPyOA', 'Status': 'ok', 'Message': None, 'User time': 0.2722933989998637, 'Wallclock time': 0.2722933989998637, 'Termination condition': 'optimal', 'Termination message': None, 'Timing': Bunch(Call after main solve = 3.056700006709434e-05, Call after subproblem solve = 1.0740001016529277e-05, OA cut generation = 0.001555258999360376, feasibility subproblem = 0.034517051999500836, fixed subproblem = 0.08473014199989848, initialization = 0.05965278199983004, main = 0.048688222999771824, main loop = 0.17458768599954055, main_timer_start_time = 7516.527777676, total = 0.2722933989998637), 'Iterations': 3, 

In [ ]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 2.4384471872707754


## setting options for mip and nlp solvers

In [ ]:
SolverFactory('mindtpy').solve(model,
                               strategy='OA',
                               time_limit=3600,
                               mip_solver='cbc',
                               mip_solver_args=dict(solver='cbc', warmstart=True),
                               nlp_solver='ipopt',
                               tee=True)

Starting MindtPy version 0.1.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 3600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: false
heuristic_nonconvex: false
init_strategy: rNLP
level_coef: 0.5
solution_limit: 10
reduce_level_coef: false
use_bb_tree_incumbent: false
sqp_la

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.9999999925051268, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of binary variables': 1, 'Number of integer variables': 0, 'Number of continuous variables': 1, 'Number of nonzeros': None, 'Sense': 'minimize', 'Number of disjunctions': 0}], 'Solver': [{'Name': 'MindtPyOA', 'Status': 'ok', 'Message': None, 'User time': 0.0804327750001903, 'Wallclock time': 0.0804327750001903, 'Termination condition': None, 'Termination message': None, 'Timing': Bunch(OA cut generation = 0.0006000469993523438, initialization = 0.05889207699965482, main = 0.0018029169996225392, main loop = 0.004857405999246112, main_timer_start_time = 8121.651330863, total = 0.0804327750001903), 'Iterations': 1, 'Num infeasible nlp subproblem': 0, 'Best solution found time': None, 'Primal integral': nan, 'Dual integral': 0.0, 'Primal dual gap integral': nan}]}